In [1]:
import pandas as pd
import numpy as np

In [2]:
from baseline_model import BaselineBEPSModel

In [3]:
pd.options.mode.chained_assignment = None

In [4]:
JAN_TARGETS_PATH = '../data/input_data/jan_proposal_emissions_targets.csv'
JUNE_TARGETS_PATH = '../data/input_data/june_proposal_emissions_targets.csv'
EMISSIONS_PATH = '../data/input_data/energy_emissions.csv'
BUILDING_DATA_PATH = '../data/input_data/cleaned_building_data_with_policy_gfa.csv'
ORIG_BUILDING_DATA_PATH = '../data/input_data/rmi_building_analysis_with_new_col_names.csv'

JAN_FINE_YEARS = [2027, 2030, 2035, 2040, 2045, 2050]
JUNE_FINE_YEARS = [2030, 2035, 2040, 2045, 2050]
FINE_PER_SQ_FT = 2.5

In [ ]:
rmi_buildings = pd.read_csv(ORIG_BUILDING_DATA_PATH)

In [ ]:
rmi_buildings[rmi_buildings['OSEBuildingID'] == 50259].iloc[0]

In [ ]:
rmi_buildings['LargestPropertyUseType OSE'].unique()

In [ ]:
timeline = pd.read_csv(JAN_TARGETS_PATH)

In [ ]:
timeline.columns

In [ ]:
row = timeline[(timeline['year'] == 2030) & (timeline['sq_ft_classification'] == 'B') & (timeline['building_type'] == np.nan)]

In [ ]:
row.iloc[0]

In [ ]:
rmi_buildings[rmi_buildings['LargestPropertyUseType OSE'].isna()][['LargestPropertyUseType OSE', 'SecondLargestPropertyUseType OSE', 'ThirdLargestPropertyUseType OSE']]

In [ ]:
sample_building = rmi_buildings[rmi_buildings['LargestPropertyUseType OSE'].isna()].iloc[0]

In [ ]:
'cow' if pd.isna(sample_building['LargestPropertyUseType OSE']) else 'moose'

So it looks like the final row in `_get_city_ghgi` should work correctly. Let's test that theory.

In [ ]:
all_nan_building_types = rmi_buildings[(rmi_buildings['LargestPropertyUseType OSE'].isna()) & (rmi_buildings['SecondLargestPropertyUseType OSE'].isna()) & (rmi_buildings['ThirdLargestPropertyUseType OSE'].isna())]

In [ ]:
all_nan_building_types

In [ ]:
all_nan_building_types.to_csv('nan_buildings.csv')

In [ ]:
nan_buildings_model = BaselineBEPSModel(EMISSIONS_PATH, JAN_TARGETS_PATH, 'nan_buildings.csv', JAN_FINE_YEARS, FINE_PER_SQ_FT)

In [ ]:
nan_buildings_model.calculate_baseline_model(2027, 2030)

In [ ]:
nan_buildings_model.scenario_results[nan_buildings_model.scenario_results['OSEBuildingID'] == 50542].iloc[0]

In [ ]:
# do any buildings report 0 energy use?

raw_buildings = pd.read_csv('../data/input_data/Reproducing RMI building data cleaning/seattle_large_building_data_2019.csv')
raw_buildings.columns

In [ ]:
raw_buildings[(raw_buildings['NaturalGas(kBtu)'] == 0) & (raw_buildings['Electricity(kWh)'] == 0) & (raw_buildings['SteamUse(kBtu)'] == 0)].iloc[0]

In [ ]:
# try to run now

In [5]:
jan_model = BaselineBEPSModel(EMISSIONS_PATH, JAN_TARGETS_PATH, ORIG_BUILDING_DATA_PATH, JAN_FINE_YEARS, FINE_PER_SQ_FT)

In [9]:
jan_model.scenario_results.head()

,OSEBuildingID,BuildingName,Total GFA for Policy,sq_ft_classification,LargestPropertyUseType OSE,SecondLargestPropertyUseType OSE,ThirdLargestPropertyUseType OSE,year,expected_baseline,expected_baseline_ghgi,city_ghgi_target,compliant_ghgi,compliant_emissions,compliance_status,compliance_fees
0,1,MAYFLOWER PARK HOTEL,88434,C,Hotel,NaN,NaN,2027.0,2.705014e+05,3.058794,3.058794,3.058794,2.705014e+05,No,221085.0
1,2,PARAMOUNT HOTEL,88502,C,Hotel,NaN,Restaurant,2027.0,2.843616e+05,3.213052,3.213052,3.213052,2.843616e+05,No,221255.0
2,3,WESTIN HOTEL (Parent Building),756493,A,Hotel,NaN,Recreation,2027.0,2.193901e+06,2.900095,2.220000,2.220000,1.679414e+06,No,1891232.5
3,5,HOTEL MAX,61320,C,Hotel,NaN,NaN,2027.0,2.934861e+05,4.786141,4.786141,4.786141,2.934861e+05,No,153300.0
4,8,WARWICK SEATTLE HOTEL,123445,B,Hotel,NaN,Recreation,2027.0,5.827520e+05,4.720742,2.220000,2.220000,2.740479e+05,No,308612.5
